### Practica 2 VISION POR COMPUTADOR TAREA 4

In [1]:
import cv2  # pip install opencv-python
import numpy as np # pip install numpy
import matplotlib.pyplot as plt # pip install matplotlib
import random
import mediapipe as mp # pip install mediapipe    o en caso de problemas    pip install mediapipe --user
import pygame  # pip install pygame

pygame 2.6.0 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


TAREA: Tras ver el video de  [Virtual air guitar](https://youtu.be/FIAmyoEpV5c?feature=shared) decidimos proponer un demostrador parecido al de Virtual air guitar pero con un piano, donde el usuario una vez ponga las dos manos en la pantalla podra tocar un piano como si fuese de verdad.

Lo primero para poder resolver esta tarea es decompnerlo en partes, por lo que lo primero es poder detectar las manos, para ello tomamos un codigo de partida, el cual modificamos ciertas partes para nuestra demostracion [Codigo de partida en Medium](https://lvimuth.medium.com/hand-detection-in-python-using-opencv-and-mediapipe-30c7b54f5ff4)

In [2]:
class HandTrackingDynamic:
    def __init__(self, mode=False, maxHands=2, detectionCon=0.5, trackCon=0.5):
        self.__mode__ = mode
        self.__maxHands__ = maxHands
        self.__detectionCon__ = detectionCon
        self.__trackCon__ = trackCon
        self.handsMp = mp.solutions.hands
        self.hands = self.handsMp.Hands(max_num_hands=maxHands)
        self.mpDraw = mp.solutions.drawing_utils
        self.tipIds = [4, 8, 12, 16, 20]  # IDs de las puntas de los dedos

    def findFingers(self, frame):
        imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        return frame

    def findPosition(self, frame):
        hands_positions = []
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                lmsList = []
                for id, lm in enumerate(handLms.landmark):
                    h, w, c = frame.shape
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    lmsList.append([id, cx, cy])

                hands_positions.append(lmsList)

                # Dibuja el contorno de la mano como un guante
                self.draw_glove(frame, lmsList)

        return hands_positions

    def draw_glove(self, frame, lmsList):
        points = [(lm[1], lm[2]) for lm in lmsList]

        # Agregar la muñeca a los puntos, ajustando su posición hacia abajo
        wrist_x = lmsList[0][1]
        wrist_y = lmsList[0][2] + 50  # Desplazar hacia abajo (ajusta este valor según sea necesario)
        points.insert(0, (wrist_x, wrist_y))  # Añadir muñeca al principio

        points = np.array(points, dtype=np.int32)

        # Dibuja el contorno de la mano como un guante
        cv2.fillPoly(frame, [points], (255, 255, 255))  # Color blanco para el guante
        cv2.polylines(frame, [points], isClosed=True, color=(255, 255, 255), thickness=10)  # Dibuja el contorno

    def findFingerUp(self, lmsList):
        fingers = []
        if lmsList[self.tipIds[0]][1] > lmsList[self.tipIds[0] - 1][1]:
            fingers.append(1)
        else:
            fingers.append(0)

        for id in range(1, 5):
            if lmsList[self.tipIds[id]][2] < lmsList[self.tipIds[id] - 2][2]:
                fingers.append(1)
            else:
                fingers.append(0)

        return fingers

Lo siguiente a tener en cuenta es que tenemos que tener los audios de piano para que el usuario pueda tocar el piano

In [3]:
def play_sound(zone, sustained=False):
    volume = random.uniform(0.5, 1.0)  # Volumen aleatorio entre 0.5 y 1.0
    sound = None

    # Seleccionar sonido dependiendo de si es sostenido
    if sustained:
        sound_files = [
            'air_piano_sounds/do_stretched.wav',
            'air_piano_sounds/re_stretched.wav',
            'air_piano_sounds/mi_stretched.wav',
            'air_piano_sounds/fa_stretched.wav',
            'air_piano_sounds/sol_stretched.wav',
            'air_piano_sounds/la_stretched.wav',
            'air_piano_sounds/si_stretched.wav',
            'air_piano_sounds/do_octave_stretched.wav'
        ]
    else:
        sound_files = [
            'air_piano_sounds/do.wav',
            'air_piano_sounds/re.wav',
            'air_piano_sounds/mi.wav',
            'air_piano_sounds/fa.wav',
            'air_piano_sounds/sol.wav',
            'air_piano_sounds/la.wav',
            'air_piano_sounds/si.wav',
            'air_piano_sounds/do_octave.wav'
        ]

    if 1 <= zone <= 8:
        sound = pygame.mixer.Sound(sound_files[zone - 1])
        sound.set_volume(volume)  # Establece el volumen
        sound.play()

Preparamos los metodos para que el usuario detecte cuando esta en modo air piano y cuando no dibujando las zonas, indicandoselo con texto y demas


In [4]:
def is_piano_posture(fingers):
    # Definimos la postura de piano como cuando los dedos índice, medio, anular y meñique están levantados
    return fingers[1] == 1 and fingers[2] == 1 and fingers[3] == 1 and fingers[4] == 1

def draw_centered_text(frame, text, font_scale, color):
    # Calcular el tamaño del texto
    (text_width, text_height), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, 2)
    
    # Calcular las coordenadas para centrar el texto
    center_x = (frame.shape[1] - text_width) // 2
    center_y = (text_height + 30)  # Colocar el texto en la parte superior

    # Dibujar el texto varias veces para simular negrita
    for dx in [-1, 0, 1]:  # Desplazamiento para simular negrita
        cv2.putText(frame, text, (center_x + dx, center_y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, 2)

def draw_zones(frame, zone_states):
    zone_width = frame.shape[1] // 8
    for i in range(8):
        x_start = i * zone_width
        x_end = (i + 1) * zone_width
        color = (50, 50, 50)  # Color base de las zonas
        if zone_states[i]:
            color = (0, 255, 0)  # Color verde para zonas activas
        cv2.rectangle(frame, (x_start, 0), (x_end, frame.shape[0]), color, 2)
        # Opcional: Añadir etiquetas de nota
        notas = ['Do', 'Re', 'Mi', 'Fa', 'Sol', 'La', 'Si', 'Do']
        cv2.putText(frame, notas[i], (x_start + 10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

Una vez todo listo vamos tomando la camara y cuando las dos manos esten juntas cada movimiento de los dedos del usuario hara sonar el piano

In [5]:
def main():
    pygame.mixer.init()  # Inicia pygame mixer para sonidos
    cap = cv2.VideoCapture(0)
    detector = HandTrackingDynamic()
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)  # Aumentar la resolución para mejor visualización
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    if not cap.isOpened():
        print("Cannot open camera")
        exit()

    # Variables para controlar sonidos sostenidos
    sustained_notes = [False] * 8  # Estado de las notas sostenidas
    zone_states = [False] * 8  # Estado visual de las zonas

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Cerrar con la tecla 'Escape'
        key = cv2.waitKey(1) & 0xFF
        if key == 27:  # Escape
            break

        frame = detector.findFingers(frame)
        hands_positions = detector.findPosition(frame)

        # Ponemos todo si tenemos dos manos
        if len(hands_positions) == 2:

            for handLms in hands_positions:
                if len(handLms) != 0:
                    # Verificar si la mano tiene la postura de "piano"
                    fingers = detector.findFingerUp(handLms)
                    
                    # Revisar si hay dedos en la postura de piano
                    if is_piano_posture(fingers):
                        # Asegúrate de que fingers tenga al menos 8 elementos
                        if len(fingers) >= 8:
                            for i in range(1, 9):
                                if fingers[i] == 1:
                                    if not sustained_notes[i - 1]:
                                        play_sound(i, sustained=True)
                                        sustained_notes[i - 1] = True  # Marcar nota como sostenida
                                        zone_states[i - 1] = True  # Activar estado visual
                                else:
                                    sustained_notes[i - 1] = False  # Resetear estado al bajar el dedo
                                    zone_states[i - 1] = False  # Desactivar estado visual
                    else:
                        # Obtener coordenadas del dedo índice (ID 8)
                        x, y = handLms[8][1], handLms[8][2]

                        # Dividir la pantalla en 8 zonas y reproducir sonido basado en la posición
                        current_zone = min(max((x // (frame.shape[1] // 8)) + 1, 1), 8)  # Cálculo más conciso de zonas
                        play_sound(current_zone)
                        zone_states[current_zone - 1] = True  # Activar estado visual

            # Añadir un fondo semi-transparente
            overlay = frame.copy()
            alpha = 0.3  # Transparencia
            cv2.rectangle(overlay, (0, 0), (frame.shape[1], 100), (0, 0, 0), -1)  # Fondo negro semi-transparente
            cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
            draw_zones(frame, zone_states)
            draw_centered_text(frame, 'Air Piano', 1, (255, 255, 255))  # Dibujar el texto

        cv2.imshow('Air Piano use your hands', frame)

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
